In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
from sqlalchemy import text

## Dataset Loading

In [3]:
businesses = pd.read_csv("Businesses.csv")
income = pd.read_csv("Income.csv")
population = pd.read_csv("Population.csv")
stops = pd.read_csv("Stops.txt")

In [4]:
businesses.info()
income.info()
population.info()
stops.info() 
# only the df stops contains null values (columns: stop_code, location_type, parent_station and platform_code)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12217 entries, 0 to 12216
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   industry_code           12217 non-null  object
 1   industry_name           12217 non-null  object
 2   sa2_code                12217 non-null  int64 
 3   sa2_name                12217 non-null  object
 4   0_to_50k_businesses     12217 non-null  int64 
 5   50k_to_200k_businesses  12217 non-null  int64 
 6   200k_to_2m_businesses   12217 non-null  int64 
 7   2m_to_5m_businesses     12217 non-null  int64 
 8   5m_to_10m_businesses    12217 non-null  int64 
 9   10m_or_more_businesses  12217 non-null  int64 
 10  total_businesses        12217 non-null  int64 
dtypes: int64(8), object(3)
memory usage: 1.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------       

In [5]:
# schools dataset
catchments_primary = gpd.read_file("Catchments/catchments_primary.shp")
catchments_future = gpd.read_file("Catchments/catchments_future.shp")
catchments_secondary = gpd.read_file("Catchments/catchments_secondary.shp")

In [6]:
catchments_primary.info()
# Add_DATE and PRIORITY contain nulls

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1662 entries, 0 to 1661
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  1662 non-null   geometry
dtypes: geometry(1)
memory usage: 13.1 KB


In [ ]:
catchments_future.info()

In [ ]:
catchments_secondary.info()

In [126]:
sa2_regions = gpd.read_file("SA2_2021_SHP/SA2_2021_AUST_GDA2020.shp")
sa2_regions.info()
#geometry and AREASQKM21 contain null values

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2473 entries, 0 to 2472
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   SA2_CODE21  2473 non-null   object  
 1   SA2_NAME21  2473 non-null   object  
 2   CHG_FLAG21  2473 non-null   object  
 3   CHG_LBL21   2473 non-null   object  
 4   SA3_CODE21  2473 non-null   object  
 5   SA3_NAME21  2473 non-null   object  
 6   SA4_CODE21  2473 non-null   object  
 7   SA4_NAME21  2473 non-null   object  
 8   GCC_CODE21  2473 non-null   object  
 9   GCC_NAME21  2473 non-null   object  
 10  STE_CODE21  2473 non-null   object  
 11  STE_NAME21  2473 non-null   object  
 12  AUS_CODE21  2473 non-null   object  
 13  AUS_NAME21  2473 non-null   object  
 14  AREASQKM21  2454 non-null   float64 
 15  LOCI_URI21  2473 non-null   object  
 16  geometry    2454 non-null   geometry
dtypes: float64(1), geometry(1), object(15)
memory usage: 328.6+ KB


## Dataset Cleaning and Transformation

In [ ]:
# Renaming the columns for latitude and longitude
stops = stops.rename(columns={'stop_lat':'lat', 'stop_lon':'lng'})

# creating the geometry column 
stops['geom'] = gpd.points_from_xy(stops.lng, stops.lat) 

# dropping the old lat and lng columns 
stops = stops.drop(columns=['lat', 'lng', 'wheelchair_boarding', 'platform_code', 'location_type'])

stops

In [ ]:
catchments_primary.columns.tolist()

catchments_primary = catchments_primary.drop(columns = ['YEAR7','YEAR8','YEAR9','YEAR10','YEAR11','YEAR12', 'PRIORITY', 'ADD_DATE'])

catchments_primary

In [ ]:
catchments_secondary.columns.tolist()

# drop irrelevant columns 
catchments_secondary = catchments_secondary.drop(columns = ['KINDERGART','YEAR1','YEAR2','YEAR3','YEAR4','YEAR5','YEAR6', 'PRIORITY', 'ADD_DATE'])

catchments_secondary

In [ ]:
sa2_regions.columns.tolist()

# drop irrelevant columns
#sa2_regions = sa2_regions.drop(columns=['CHG_FLAG21','CHG_LBL21'])

# drop rows with missing values
sa2_regions = sa2_regions.dropna(subset=['geometry', 'AREASQKM21'])

# filter for Greater Sydney
sa2_regions = sa2_regions[sa2_regions['GCC_NAME21']=='Greater Sydney']
sa2_regions # only 373 entries in Greater Sydney

In [216]:
# A list of SA4 names for you guys to choose from
sa2_regions.SA4_NAME21.unique().tolist()

sa2_regions[sa2_regions['SA4_NAME21'] == 'Sydney - Eastern Suburbs']
# I'm doing Eastern suburbs

,SA2_CODE21,SA2_NAME21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
370,118011339,Bondi - Tamarama - Bronte,11801,Eastern Suburbs - North,118,Sydney - Eastern Suburbs,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,2.4962,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.25848 -33.89114, 151.25858 -33.8..."
371,118011340,Bondi Beach - North Bondi,11801,Eastern Suburbs - North,118,Sydney - Eastern Suburbs,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,2.4604,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.26933 -33.88297, 151.26955 -33.8..."
372,118011341,Bondi Junction - Waverly,11801,Eastern Suburbs - North,118,Sydney - Eastern Suburbs,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,2.5235,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.24094 -33.89472, 151.24095 -33.8..."
373,118011342,Centennial Park,11801,Eastern Suburbs - North,118,Sydney - Eastern Suburbs,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,1.9786,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.23776 -33.89064, 151.23874 -33.8..."
374,118011344,Dover Heights,11801,Eastern Suburbs - North,118,Sydney - Eastern Suburbs,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,1.8705,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.27825 -33.86109, 151.27831 -33.8..."
375,118011345,Paddington - Moore Park,11801,Eastern Suburbs - North,118,Sydney - Eastern Suburbs,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.7250,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.21837 -33.88423, 151.21848 -33.8..."
376,118011346,Rose Bay - Vaucluse - Watsons Bay,11801,Eastern Suburbs - North,118,Sydney - Eastern Suburbs,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,5.1368,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((151.25862 -33.85965, 151.25865..."
377,118011347,Woollahra,11801,Eastern Suburbs - North,118,Sydney - Eastern Suburbs,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,1.2805,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.25351 -33.88693, 151.25379 -33.8..."
378,118011649,Bellevue Hill,11801,Eastern Suburbs - North,118,Sydney - Eastern Suburbs,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.1556,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.24488 -33.87712, 151.24496 -33.8..."
379,118011650,Double Bay - Darling Point,11801,Eastern Suburbs - North,118,Sydney - Eastern Suburbs,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,1.5591,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.245 -33.87377, 151.24515 -33.873..."


## Ingestion into SQL  

In [142]:
srid = 4326
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
stops

,stop_id,stop_code,stop_name,parent_station,geom
0,200039,200039.0,"Central Station, Eddy Av, Stand A",200060,POINT (151.20666465471 -33.8822064874687)
1,200054,200054.0,"Central Station, Eddy Av, Stand D",200060,POINT (151.20699145565 -33.8820421431408)
2,200060,NaN,Central Station,NaN,POINT (151.206292455081 -33.8840842535493)
3,201510,NaN,Redfern Station,NaN,POINT (151.198866071817 -33.8916900512711)
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",201510,POINT (151.198881722942 -33.8933293130144)
...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",21271,POINT (151.07879697831 -33.8220164586429)
114714,2137185,2137185.0,"Cabarita Wharf, Side A",21371,POINT (151.116926480557 -33.8406690716775)
114715,2137186,2137186.0,"Cabarita Wharf, Side B",21371,POINT (151.116898892402 -33.8407691073139)
114716,21501,21501.0,Parramatta Wharf,2150112,POINT (151.010576673346 -33.8139042429414)


In [148]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

def transform_spatial_col(df:pd.DataFrame, spatial_col:str,srid:int):
    df_og = df.copy()  # creating a copy of the original for later
    df['geom'] = df[spatial_col].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the create_wkt_element
    df = df.drop(columns=spatial_col)  # deleting the old copy
    return df 

In [ ]:
transform_spatial_col(catchments_primary, 'geometry', 4326)

In [ ]:
transform_spatial_col(catchments_future, 'geometry', 4326)

In [150]:
transform_spatial_col(catchments_secondary, 'geometry', 4326)

,USE_ID,CATCH_TYPE,USE_DESC,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,geom
0,8503,HIGH_COED,Billabong HS,Y,Y,Y,Y,Y,Y,MULTIPOLYGON (((146.67182402032344 -35.3144375...
1,8266,HIGH_COED,James Fallon HS,Y,Y,Y,Y,Y,Y,MULTIPOLYGON (((147.08733806259178 -35.8627146...
2,8505,HIGH_COED,Murray HS,Y,Y,Y,Y,Y,Y,MULTIPOLYGON (((146.81447829547324 -35.7834062...
3,8458,HIGH_COED,Kingswood HS,Y,Y,Y,Y,Y,Y,MULTIPOLYGON (((150.68599834118749 -33.7403060...
4,8559,HIGH_COED,Jamison HS,Y,Y,Y,Y,Y,Y,MULTIPOLYGON (((150.69513440644116 -33.7562688...
...,...,...,...,...,...,...,...,...,...,...
431,8213,HIGH_BOYS,Birrong BHS,Y,Y,Y,Y,Y,Y,MULTIPOLYGON (((151.05363508494244 -33.8507612...
432,8108,HIGH_COED,Cessnock HS,Y,Y,Y,Y,Y,Y,MULTIPOLYGON (((151.4285214252146 -32.74415247...
433,3235,CENTRAL_HIGH,Tooleybuc CS,Y,Y,Y,Y,Y,Y,MULTIPOLYGON (((143.37723402388323 -34.8017260...
434,1115,CENTRAL_HIGH,Balranald CS,Y,Y,Y,Y,Y,Y,MULTIPOLYGON (((143.65540852703225 -33.5570248...


In [204]:

dict(population.dtypes)


{'sa2_code': dtype('int64'),
 'sa2_name': dtype('O'),
 '0-4_people': dtype('int64'),
 '5-9_people': dtype('int64'),
 '10-14_people': dtype('int64'),
 '15-19_people': dtype('int64'),
 '20-24_people': dtype('int64'),
 '25-29_people': dtype('int64'),
 '30-34_people': dtype('int64'),
 '35-39_people': dtype('int64'),
 '40-44_people': dtype('int64'),
 '45-49_people': dtype('int64'),
 '50-54_people': dtype('int64'),
 '55-59_people': dtype('int64'),
 '60-64_people': dtype('int64'),
 '65-69_people': dtype('int64'),
 '70-74_people': dtype('int64'),
 '75-79_people': dtype('int64'),
 '80-84_people': dtype('int64'),
 '85-and-over_people': dtype('int64'),
 'total_people': dtype('int64')}

In [178]:
conn.rollback()

conn.execute(text("""
DROP TABLE IF EXISTS businesses;
CREATE TABLE businesses (
    industry_code INTEGER,
    industry_name VARCHAR(100),
    sa2_code INTEGER, 
    sa2_name VARCHAR(100),
    0_to_50k_businesses INTEGER,
    50k_to_200k_businesses INTEGER,
    200k_to_2m_businesses INTEGER,
    2m_to_5m_businesses INTEGER,
    5m_to_10m_businesses INTEGER,
    10m_or_more_businesses INTEGER,
    total_businesses INTEGER
);"""
))


conn.execute(text("""
DROP TABLE IF EXISTS income;
CREATE TABLE income (
    sa2_code21 INTEGER,
    sa2_name VARCHAR(100),
    earners INTEGER, 
    median_age INTEGER,
    median_income INTEGER,
    mean_income INTEGER
);"""
))

conn.execute(text("""
DROP TABLE IF EXISTS income;
CREATE TABLE income (
    sa2_code21 INTEGER,
    sa2_name VARCHAR(100),
    earners INTEGER, 
    median_age INTEGER,
    median_income INTEGER,
    mean_income INTEGER
);"""
))


conn.execute(text("""
DROP TABLE IF EXISTS stops;
CREATE TABLE stops (
    stopid INTEGER UNIQUE,  
    stop_code INTEGER, 
    stop_name VARCHAR(100),
    parent_station INTEGER,
    geom GEOMETRY(POINT,4326)
);"""
))

In [152]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [154]:
db, conn = pgconnect(credentials)

Connected successfully.


In [156]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.5 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [202]:
conn.rollback()
income.to_sql("income", conn, if_exists='append', index=False)
query(conn, "select * from income")

,sa2_code21,sa2_name,earners,median_age,median_income,mean_income
0,101021007,Braidwood,2467,51,46640,68904
1,101021008,Karabar,5103,42,65564,69672
2,101021009,Queanbeyan,7028,39,63528,69174
3,101021010,Queanbeyan - East,3398,39,66148,74162
4,101021012,Queanbeyan West - Jerrabomberra,8422,44,78630,91981
...,...,...,...,...,...,...
1921,128021537,Royal National Park,14,37,36980,47584
1922,128021538,Sutherland - Kirrawee,13895,41,64940,74867
1923,128021607,Engadine,10239,43,63695,72995
1924,128021608,Loftus - Yarrawarrah,4424,45,63087,76440


In [ ]:
stops.to_sql('stops', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})
query(conn, "select * from stops")

In [218]:
target_url = 'https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer'

In [ ]:
conn.close()
db.dispose()